In [4]:
# Initialize Otter
import otter
grader = otter.Notebook("assignment6.ipynb")

# DS 453 / 653: Programming Assignment 6

**Due date**: Sunday, March 31 at 8pm on [Gradescope](https://www.gradescope.com/courses/710247).

_You must follow the Academic Code of Conduct and Collaboration Policy stated in the course syllabus at all times while working on this assignment._

This assignment contains 3 questions worth a total of 5 points. You must receive at least 4 points to pass the assignment.

To begin, please execute the code block below:

In [5]:
import otter
grader = otter.Notebook()

In [6]:
# Execute this block only if you are using Google Colab.
# If you are running the notebook file locally, install pycryptodome yourself but do NOT install the package pycrypto.

%pip install pycryptodome

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\jackw\appdata\local\programs\python\python311\lib\site-packages\cloudpickle-1.6.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\jackw\appdata\local\programs\python\python311\lib\site-packages\gym-0.19.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\jackw\appdata\local\programs\python\python311\lib\site-packages\luxai2021-0.1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\jackw\

## Assignment Overview



#### Helper functions

In [7]:
from Crypto.Util.strxor import strxor
from binascii import hexlify, unhexlify

import string
import os

# the one-time pad operation, applied to bytestrings for the message and key
def one_time_pad(bytesMessage, bytesKey):
    """ If both the message and key are of type string and have the same length,
        then takes their strxor and returns the corresponding ciphertext.
    """
    assert(type(bytesMessage) == bytes)
    assert(type(bytesKey)     == bytes)
    assert(len(bytesMessage)  == len(bytesKey))
    return strxor(bytesMessage, bytesKey)

# list of the top 100 words, in all lowercase letters
words_lower = [b'a', b'about', b'after', b'all', b'also', b'an', b'and', b'any', b'as',
               b'at', b'back', b'be', b'because', b'but', b'by', b'can', b'come',
               b'could', b'day', b'do', b'even', b'first', b'for', b'from', b'get',
               b'give', b'go', b'good', b'have', b'he', b'her', b'him', b'his', b'how',
               b'i', b'if', b'in', b'into', b'it', b'its', b'just', b'know', b'like',
               b'look', b'make', b'me', b'most', b'my', b'new', b'no', b'not', b'now',
               b'of', b'on', b'one', b'only', b'or', b'other', b'our', b'out', b'over',
               b'people', b'say', b'see', b'she', b'so', b'some', b'take', b'than',
               b'that', b'the', b'their', b'them', b'then', b'there', b'these', b'they',
               b'think', b'this', b'time', b'to', b'two', b'up', b'us', b'use', b'want',
               b'way', b'we', b'well', b'what', b'when', b'which', b'who', b'will',
               b'with', b'work', b'would', b'year', b'you', b'your']

# top 100 words, this time with a capitalized first letter
words_upper = [(word.decode('ascii')[0].upper() + word.decode('ascii')[1:]).encode('ascii') for word in words_lower]

# subset of the above list that happens to be three-letter words
three_letter_words_lower = [b'all', b'and', b'any', b'but', b'can', b'day', b'for',
                           b'get', b'her', b'him', b'his', b'how', b'its', b'new',
                           b'not', b'now', b'one', b'our', b'out', b'say', b'see',
                           b'she', b'the', b'two', b'use', b'way', b'who', b'you']

# the capitalized version of the three letter words list
three_letter_words_upper = [(word.decode('ascii')[0].upper() + word.decode('ascii')[1:]).encode('ascii') for word in three_letter_words_lower]

### Question 1: Two-time pad attack on three-character lowercase ciphertexts (2 points)

A one-time pad simply involves the xor of a message with a key to produce a ciphertext: `c = m ^ k`, where `^` denotes the XOR operation. It is essential that the key be as long as the message, or in other words that the key not be repeated for two distinct message blocks.

In this problem, you'll see why the key must only be used once. Your goal is to break small two-time-pad examples. The input ciphertexts are created by:

- Choosing two plaintexts at random from the list `three_letter_words_lower`. (This list contains the 3-letter words from the set of the 100 most common English words listed on [Wikipedia](https://en.wikipedia.org/wiki/Most_common_words_in_English), in lowercase format.)

- Applying the `one_time_pad()` function to both words, using the same 3-letter `bytesKey` (which is *not* provided!).

__Your task__: Even though you don't know the secret key, your job is to recover the two inital messages and return them as a set. (Hint: the messages are short enough that a brute-force attack can work here.)

- Input: `c1`, `c2` (byte objects): Ciphertexts created by xor-ing a 3-letter word with some random 3-letter key

- Return: `{m1, m2}`: A set of two different `three_letter_words` corresponding to the original plaintext messages


__Your response:__

In [8]:
print(one_time_pad(b'day', b'zzz'))

b'\x1e\x1b\x03'


In [9]:
def q1_short_lower_twotimepad(c1, c2) -> set:
    """ Returns the set of possible plaintexts for the two-time pad ciphertexts c1 and c2,
        assuming that the plaintexts are short and lowercase.
    """
    output = set()
    for word1 in three_letter_words_lower:
        for word2 in three_letter_words_lower:
            key = one_time_pad(word1, c1)
            if one_time_pad(word2, key) == c2:
                output.add(word1)
                output.add(word2)
    return output    



In [10]:
q1_short_lower_twotimepad(b'\x1e\x1b\x03', b'\x0f\t\x1f')

{b'day', b'use'}

Here is a particular test you can use to check whether your code works. Given the ciphertexts:
```
c1 = b'\x1e\x1b\x03'
c2 = b'\x0f\t\x1f'
```

the corresponding two plaintexts should be `{b'day', b'use'}`, with the key `k = b'zzz'`.

_Points:_ 2

In [11]:
grader.check("q1")

q1 results: All test cases passed!

### Question 2: Two-time pad attack with capitalized and lowercase words (1 point)

Now, we make a small change to the previous question. This time, we guarantee that:
- the first message `m1` is a capitalized word from the list `three_letter_words_upper` (e.g., the word `New`).
- the second message `m2` is a lowercase word from the list `three_letter_words_lower` (e.g., the word `day`).

As a result, when you recover the two messages, you should also know their order. So, you can concatenate the two words together and put a space between them (e.g., `New day`). The one downside is that this function might return multiple possible outputs. Consider the following example:

In [12]:
print(one_time_pad(b'New', b'abc'), one_time_pad(b'day', b'abc'))
print(one_time_pad(b'Day', b'kfm'), one_time_pad(b'new', b'kfm'))

b'/\x07\x14' b'\x05\x03\x1a'
b'/\x07\x14' b'\x05\x03\x1a'


Hence, from the two ciphertexts `c1 = b'/\x07\x14'` and `c2 = b'\x05\x03\x1a'`, it might be the case that the original message was `New day` or `Day new`; you cannot distinguish between the two. Still though, this attack is deemed to be successful if we can narrow down the set of possible messages to just a small number of possible options.

__Your task__: Once again, recover the two initial messages. This time, you are allowed to output a set of possible concatenated bytestrings, as long as this set is not too large.

- Input: `c1`, `c2` (byte objects): Ciphertexts created by xor-ing a 3-letter word with some random 3-letter key. It is guaranteed that message `m1` has a capitalized first letter, whereas message `m2` does not.

- Return: A set of up to 10 possible `m1 + b' ' + m2` combinations of possible two-word plaintext messages. (Note that each individual string should be 7 bytes long, with the space in between the two 3-letter words.)

__Your response:__

_Points:_ 1

In [13]:
def q2_short_twotimepad(c1, c2) -> set:
    # TODO
    output = set()
    for word1 in three_letter_words_upper:
        for word2 in three_letter_words_lower:
            key = one_time_pad(word1, c1)
            if one_time_pad(word2, key) == c2:
                output.add(word1 + b' ' + word2)       
    return output    

In [14]:
q2_short_twotimepad(b'/\x07\x14', b'\x05\x03\x1a')

{b'Day new', b'New day'}

In [15]:
grader.check("q2")

q2 results: All test cases passed!

### Question 3: Two-time pad attack on longer messages (2 points)

__Your task:__ In this problem you will again break a cipher when the one-time pad is re-used. This time though, we have encrypted two long bytestrings that contain __multiple words__, with spaces in between the words. You are given two _hex-encoded_ ciphertexts `c1_hex` and `c2_hex` that have the same length as each other (note that the particular length is not fixed). They were formed by applying a “one-time pad” to two different messages _with the same key_. You must find the two corresponding messages `m_1` and `m_2`.

_Note: Do not try a brute-force attack on the key! That will quickly turn into "energy of the sun" type of work; in particular, you won't be able to complete the homework assignment by the deadline._

To make your search for the plaintext messages simpler, let me lay out a few ground rules:

1. Each message consists of English words in ASCII separated by spaces. No punctuation appears in the messages. 

2. Every character in the text is either a lowercase letter or a space, except that the first character of `m_1` is capitalized. The remaining words of `m_1` are lowercase, and all words in `m_2` are lowercase.

3. The second message m_2 starts with a space.

4. All of the words within each message is guaranteed to come from the set `words_lower` of the [100 most common English words](https://en.wikipedia.org/wiki/Most_common_words_in_English), except that the first message starts with a word in `words_upper`.

There are two ways that you can solve this question:

1. Write code yourself.
2. Use the interactive two-time pad cracking tool at https://www.tausquared.net/pages/ctf/twotimepad.html

__Test vector:__ To help you understand the question, here is a test vector:

```
q3_fulltwo_time_pad(b'3e21aa44d5319f099129e1859562eaeb26f558846e7dccbef7dd1a46e94d', b'5033b844dc318d4cd732fbc0d463e9b822b05892226c84bfeb8a4e4ce546')
```

should return `{'New day for all people how can we make the most of this time'}`.  In this test vector, note that:
- message 1 is 'New day for all people how can',
- message 2 is ' we make the most of this time',
- each message is 30 bytes long, and
- the first message starts with an uppercase letter, and the second message starts with a space.

Before writing any code, try using the interactive two-time pad cracking tool to find this message. Copy/paste the two hex ciphertexts, then guess "New" for the start of the first message and " we" for the start of the second message. Note how the cracking tool shows that they match. As you add a new word to each message, it shows possible candidates for the word on the other message.

__Your response:__

In [23]:
def q3_full_two_time_pad(c1_hex, c2_hex) -> set:
    ''' Input: Two hex-encoded ciphertexts c1_hex and c2_hex
        Return: Up to 100 possible m1 + m2 combinations of possible plaintext messages.
                Since m2 already starts with a space in this question,
                do *not* add an additional space in between m1 and m2.
    '''

    c1 = unhexlify(c1_hex)
    c2 = unhexlify(c2_hex)

    # TODO
    output = set()
    output.add(b'New day for all people how can we make the most of this time')
    output.add(b'After all this time you can see me now')
    return output

You can try to test out your code on the test vector that we provided above.

In [24]:
q3_full_two_time_pad(b'3e21aa44d5319f099129e1859562eaeb26f558846e7dccbef7dd1a46e94d', b'5033b844dc318d4cd732fbc0d463e9b822b05892226c84bfeb8a4e4ce546')

{b'After all this time you can see me now',
 b'New day for all people how can we make the most of this time'}

In [25]:
def test3_0():
    answerstring = b'New day for all people how can we make the most of this time'
    return answerstring in q3_full_two_time_pad(b'3e21aa44d5319f099129e1859562eaeb26f558846e7dccbef7dd1a46e94d',
                                                b'5033b844dc318d4cd732fbc0d463e9b822b05892226c84bfeb8a4e4ce546')
test3_0()

True

We provide two tests for this question, each one of which is worth 1 point.

- You will earn one point for solving the following two-time-pad challenge.
    - `m1 = b'5a2e8da8d55eb6b738139bda26471b26f5fa4f'`
    - `m2 = b'3b3196b8871db6b574408ad76f595e72f2f85d'`
- You will earn another point if your code works on arbitrary ciphertext pairs.

You can hardcode the answer in the function `q3_full_two_time_pad` if you want to receive only one point for this question. Even if you choose this option, make sure to show your work and explain how you found the answer!

_Points:_ 2

In [26]:
grader.check("q3")

q3 results:
    q3 - 1 result:
        ✅ Test case passed

    q3 - 2 result:
        ❌ Test case failed
        Trying:
            from binascii import hexlify
        Expecting nothing
        ok
        Trying:
            import random
        Expecting nothing
        ok
        Trying:
            from os import urandom
        Expecting nothing
        ok
        Trying:
            def one_time_pad(bytesMessage, bytesKey):
                """ If both the message and key are of type string and have the same length,
                    then takes their strxor and returns the corresponding ciphertext.
                """
                assert type(bytesMessage) == bytes
                assert type(bytesKey) == bytes
                assert len(bytesMessage) == len(bytesKey)
                return strxor(bytesMessage, bytesKey)
        Expecting nothing
        ok
        Trying:
            def generate_n_char_sentence(n, words, isBlank=False):
                if isBlank:
                    cap = n - 1
                else:
                    cap = n
                while True:
                    sentence = ''
                    while len(sentence) < cap:
                        word = random.choice(words)
                        if len(sentence) + len(word) + (1 if sentence else 0) <= cap:
                            sentence += (' ' if sentence else '') + word
                        else:
                            break
                    if len(sentence) == cap:
                        if isBlank:
                            return ' ' + sentence
                        return sentence
        Expecting nothing
        ok
        Trying:
            def test3_2():
                words = 'a about after all also an and any as at back be because but by can\n    come could day do even first for from get give go good have he her him his how\n    i if in into it its just know like look make me most my new no not now of on\n    one only or other our out over people say see she so some take than that the\n    their them then there these they think this time to two up us use want way we\n    well what when which who will with work would year you your'.split()
                cnt = 0
                total_length = 16
                total_count = 10
                for i in range(total_count):
                    m1 = generate_n_char_sentence(total_length, words, False).capitalize().encode('ascii')
                    m2 = generate_n_char_sentence(total_length, words, True).encode('ascii')
                    tempkey = urandom(total_length)
                    c1_hex = hexlify(one_time_pad(m1, tempkey))
                    c2_hex = hexlify(one_time_pad(m2, tempkey))
                    theset = q3_full_two_time_pad(c1_hex, c2_hex)
                    if m1 + m2 in theset and len(theset) <= 100:
                        cnt += 1
                    else:
                        print('Failed test on ciphertexts:', c1_hex, 'and', c2_hex, 'expected to see m1 =', m1, 'and m2 =', m2)
                return cnt / total_count >= 0.8
        Expecting nothing
        ok
        Trying:
            test3_2()
        Expecting:
            True
        **********************************************************************
        Line 49, in q3 1
        Failed example:
            test3_2()
        Expected:
            True
        Got:
            Failed test on ciphertexts: b'514753285f53f5fe1a35a7c6b24b6ef8' and b'264a4567551ff1f4562defcdfd0374ff' expected to see m1 = b'Way also the how' and m2 = b' look well no up'
            Failed test on ciphertexts: b'7d7fecbba76b31f1b46c2bc19407b71b' and b'0a7efba7a7683feef1232c8d8752a300' expected to see m1 = b'Work not only go' and m2 = b' new make i just'
            Failed test on ciphertexts: b'a5bb1aa2a9a00ebbd680695b3baaff16' and b'd2bd1cbaa9bc0fb798c9780d32abb01c' expected to see m1 = b'Work than even a' and m2 = b' its him it look'
            Failed test on ciphertexts: b'0a

## Submitting the Assignment

Please follow these instructions to complete the assignment and submit it for credit.

**Documenting collaborators, sources, and AI tools:** In accordance with the collaboration policy, use the space below to report if you used any resources to complete this homework assignment, aside from the lecture notes and the course textbooks/videos. Specifically, please report:

1. Names of all classmates you worked with, and a short description of the work that you performed together.
2. All written materials that you used, such as books or websites (besides the lecture notes or textbooks). Please include links to any web-based resources, or citations to any physical works.
3. All code that you used from other sources. In particular, if you used an AI tool, then you must include the entire exchange with the AI tool, as per the [CDS Generative AI Assistance Policy](https://www.bu.edu/cds-faculty/culture-community/gaia-policy/).

Remember that if we discover any undocumented collaborators, sources, or AI tools then this is grounds for a grade penalty and referral to BU's Academic Conduct Committee (as described in the syllabus).

_Your response:_

1.

2.

3.

**Sending to Gradescope:** After completing the assignment:
- if you did the assignment on Colab, download it in `.ipynb` format.
- if you did the assignment locally on your machine, all you need to do is to find it in your directory.

Then, submit only the `.ipynb` file to this week's programming assignment on Gradescope. It may take a few seconds or a minute for the auto-grading system to check your work.

Remember that you can submit as many times as you want until the deadline for the assignment; only your last score counts.

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Submit your assignment on Gradescope once you have passed enough tests to receive at least 4 of the 5 available points.

In [20]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)

c:\Users\jackw\AppData\Local\Programs\Python\Python311\Lib\site-packages\otter\check\notebook.py:470: UserWarning: Could not locate a PDF to include
  warnings.warn("Could not locate a PDF to include")


UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 14646: character maps to <undefined>